# Notebook 6: Report Synthesizer Agent - Development and Testing

### Objective
This notebook is for developing and testing our final agent: the `Report Synthesizer`. This agent's role is to take all the structured data generated by the previous agents in the pipeline and synthesize it into a single, human-readable, and actionable career development report in Markdown format.

In [8]:
import sys
import os
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from typing import List, Dict
import json
from IPython.display import display, Markdown

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


from src.agents import report_synthesizer

# Simulate the outputs from all previous agents

target_role = "Software Engineering"

user_skills = [
    'python', 'sql', 'java', 'git', 'data structures', 'algorithms',
    'debugging', 'problem-solving', 'communication', 'teamwork'
]


skill_gaps = [
    'kubernetes',
    'go',
    'ci/cd' # Including the one that failed to get a good test case
]


learning_plan = {
    'kubernetes': [
        {'title': 'Official Kubernetes Documentation', 'url': 'https://kubernetes.io/docs/', 'resource_type': 'Official Docs'},
        {'title': 'Kubernetes Tutorials Hub', 'url': 'https://kubernetes.io/docs/tutorials/', 'resource_type': 'Tutorials Hub'},
        {'title': 'Kubernetes YouTube Channel', 'url': 'https://www.youtube.com/results?search_query=learn+kubernetes', 'resource_type': 'Video Search'}
    ],
    'go': [
        {'title': 'Official Go Documentation', 'url': 'https://golang.org/doc/', 'resource_type': 'Official Docs'},
        {'title': 'Go Tutorials by Google', 'url': 'https://tour.golang.org/list', 'resource_type': 'Tutorials Hub'},
        {'title': 'Go Programming Language YouTube Channel', 'url': 'https://www.youtube.com/results?search_query=learn+go', 'resource_type': 'Video Search'}
    ],
    'ci/cd': [] 
}

## Developing the Report Synthesizer Agent

This agent's task is to synthesize all the data we've gathered (`target_role`, `user_skills`, `skill_gaps`, and `learning_plan`) into a single, well-structured Markdown report. The core of this agent will be a sophisticated prompt that guides the LLM to act as a career coach and content formatter.

In [12]:
def generate_report(
    target_role: str,
    user_skills: List[str],
    skill_gaps: List[str],
    learning_plan: Dict[str, List[Dict]]
) -> str:
    """
    Generates a final, human-readable report in Markdown format using an LLM.
    """
    user_skills_str = ", ".join(user_skills)
    skill_gaps_str = ", ".join(skill_gaps)
    learning_plan_str = json.dumps(learning_plan, indent=2)

    prompt_template = """
    You are an expert career coach and AI assistant. Your task is to generate a personalized, encouraging, and actionable career development report in Markdown format.

    Here is the data for the user:
    - **Target Role**: {target_role}
    - **User's Current Skills**: {user_skills}
    - **Identified Skill Gaps**: {skill_gaps}
    - **Curated Learning Plan (JSON format)**:
    ```json
    {learning_plan}
    ```

    **REPORT STRUCTURE AND INSTRUCTIONS:**

    Generate a complete report following this exact Markdown structure. Be encouraging and professional.

    1.  **Header**: Start with a main title: `# Your Personalized Career Plan`.
    2.  **Introduction**: Write a short, personalized introduction. Address the user's goal of becoming a `{target_role}` and explain that this report outlines their strengths and a clear path forward.
    3.  **Your Current Strengths**: Create a section `## Your Current Strengths`. List the `User's Current Skills` in a Markdown bulleted list. Add a brief, positive sentence acknowledging their solid foundation.
    4.  **Your Development Roadmap**: Create a section `## Your Development Roadmap`.
        -   Start with a sentence explaining that these are the key skills to focus on to bridge the gap.
        -   List the `Identified Skill Gaps` in a Markdown bulleted list.
    5.  **Your Personalized Learning Plan**: Create a main section `## Your Personalized Learning Plan`.
        -   For **each skill** found in the `Identified Skill Gaps` data:
            -   Create a sub-header formatted like: `### Learning: [Skill Name]` (e.g., `### Learning: Kubernetes`).
            -   Look at the `Curated Learning Plan` for that skill.
            -   If there are resources, list them in a bulleted list, formatted exactly like this: `- **[Resource Title](Resource URL)** - *Resource Type*`
            -   If there are **no resources** for a skill, write a sentence like: `- *No specific free resources were automatically found for this skill, but a search on platforms like YouTube or official documentation is recommended.*`
    6.  **Conclusion**: Create a final section `## Next Steps`. Write a short, motivating paragraph encouraging the user to start with one skill and continue their learning journey.

    Do not add any text or explanation before the `# Your Personalized Career Plan` header or after the conclusion.
    """

    llm = OllamaLLM(model="llama3:8b", base_url="http://127.0.0.1:11434", temperature=0.2)
    
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["target_role", "user_skills", "skill_gaps", "learning_plan"]
    )

    chain = prompt | llm

    try:
        print("Agent: Synthesizing the final report...")
        report_markdown = chain.invoke({
            "target_role": target_role,
            "user_skills": user_skills_str,
            "skill_gaps": skill_gaps_str,
            "learning_plan": learning_plan_str
        })
        print("Agent: Report generated successfully.")
        return report_markdown
    except Exception as e:
        print(f"--- AGENT ERROR ---")
        print(f"An exception occurred during report generation. Details: {e}")
        return f"# Error\n\nAn error occurred while generating the report: {e}"

## Executing the Agent and Displaying the Report

Now, let's call our new function with all the simulated data we've prepared. The output will be a single string containing the full report in Markdown. 

In [13]:
final_report = generate_report(
    target_role=target_role,
    user_skills=user_skills,
    skill_gaps=skill_gaps,
    learning_plan=learning_plan
)


display(Markdown(final_report))

Agent: Synthesizing the final report...
Agent: Report generated successfully.


Here is your personalized career development report in Markdown format:

# Your Personalized Career Plan

As you aim to become a Software Engineer, this report outlines your strengths and provides a clear path forward. You're already well-equipped with skills like Python, SQL, Java, Git, data structures, algorithms, debugging, problem-solving, communication, and teamwork. This solid foundation will serve as the base for your future growth.

## Your Current Strengths
* python
* sql
* java
* git
* data structures
* algorithms
* debugging
* problem-solving
* communication
* teamwork

Your current skills demonstrate a strong understanding of programming fundamentals, which is essential for a Software Engineer. You're well-positioned to build upon this foundation and expand your expertise.

## Your Development Roadmap
To bridge the gap between your current skills and the role of Software Engineering, focus on developing the following key skills:

* kubernetes
* go
* ci/cd

## Your Personalized Learning Plan

### Learning: Kubernetes
- **Official Kubernetes Documentation** - Official Docs
- **Kubernetes Tutorials Hub** - Tutorials Hub
- **Kubernetes YouTube Channel** - Video Search

### Learning: Go
- **Official Go Documentation** - Official Docs
- **Go Tutorials by Google** - Tutorials Hub
- **Go Programming Language YouTube Channel** - Video Search

### Learning: ci/cd
*No specific free resources were automatically found for this skill, but a search on platforms like YouTube or official documentation is recommended.*

## Next Steps

Now that you have a clear roadmap and personalized learning plan, it's time to take the first step. Start by focusing on one of the identified skills, such as Kubernetes or Go. Begin with the resources listed in your learning plan, and gradually build upon your knowledge. Remember, consistent effort and persistence will help you bridge the gap between your current skills and the role of Software Engineering. You got this!

---------------------------------------

## Final Step: Modularize the Agent

The development and validation of our `Report Synthesize` agent are now complete. The final step is to move the complete logic into its official project file, `src/agents/report_synthesizer.py`.